# Many-body potentials

`matscipy` implements support for generic manybody potentials. The class `matscipy.calculators.manybody.Manybody` implements the functional form

$$
  U
  =
  \frac{1}{2}
  \sum_{\substack{ij\\ i\neq j}}
  U_2(r^2_{ij}) + U_\text{m}(r^2_{ij}, \xi_{ij})
$$

with

$$
  \xi_{ij} 
  = 
  \sum_{\substack{k\\ k\neq i,j}} 
  \Xi(r^2_{ij}, r^2_{ik}, r^2_{jk})
$$

as described, e.g. by [Müser et al.](https://doi.org/10.1080/23746149.2022.2093129) and [Grießer et al.](https://doi.org/10.48550/arXiv.2302.08754). On top of energies and forces, the calculator can compute second derivatives (with respect to positions and strain degrees of freedom). Explicit functional forms of $U_2$, $U_\text{m}$ and $\Xi$ are implemented for a number of potential.

## Kumagai potential

The following example compute the elastic constants of a small representation of amorphous silicon using the potential by [Kumagai et al.](https://doi.org/10.1016/j.commatsci.2006.07.013).

In [23]:
import numpy as np

from ase.io import read
from ase.units import GPa

from matscipy.elasticity import elastic_moduli, full_3x3x3x3_to_Voigt_6x6
from matscipy.calculators.manybody import Manybody
from matscipy.calculators.manybody.explicit_forms import Kumagai
from matscipy.calculators.manybody.explicit_forms.kumagai import Kumagai_Comp_Mat_Sci_39_Si

# Load a disordered configuration (amorphous silicon)
a = read('../../tests/aSi.cfg')

# Construct calculator
calc = Manybody(**Kumagai(Kumagai_Comp_Mat_Sci_39_Si))
a.calc = calc

# Total energy
print(f'Cohesive energy = {a.get_potential_energy() / len(a)} eV/atom')

# Born elastic constants (without nonaffine displacements)
C = calc.get_born_elastic_constants(a)

# Get isotropic elastic moduli
E, nu, G, B, K = elastic_moduli(full_3x3x3x3_to_Voigt_6x6(C))

print(f"Young's modulus = {E.mean() / GPa} GPa")
print(f"Poisson number = {(nu + np.eye(3)).sum()/6}")

Cohesive energy = -4.415832408778078 eV
Young's modulus = 170.88211399586595 GPa
Poisson number = 0.21150737791390495


## Other many-body potentials

To runs the same code with [Stillinger-Weber](https://doi.org/10.1103/PhysRevB.31.5262), replace the calculator by

In [24]:
from matscipy.calculators.manybody.explicit_forms import StillingerWeber
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import Stillinger_Weber_PRB_31_5262_Si

calc = Manybody(**StillingerWeber(Stillinger_Weber_PRB_31_5262_Si))

Other examples of potentials that are implemented are [Tersoff](https://doi.org/10.1103/PhysRevB.39.5566), [Brenner](https://doi.org/10.1103/PhysRevB.42.9458) (without the lookup tables), [Erhart-Albe](https://doi.org/10.1103/PhysRevB.71.035211) and others. Second derivatives are supported for all of these.